Код обрабатывает изображения исходного датасета.  
Предполагается, что все изображения изначально в правильной ориентации, т.е. угол поворота - 0 (исправлено вручную).  
Разделение на трейн, валидацию и тест в соотношении 64:16:20.  
В каждом сете изображения поворачивает на 90, 180 и 270 градусов. Сохраняет в поддиректорую в соответствии с меткой класса. Таким образом, количество изображений в тренировочной выборке увеличивается в 4 раза.  

### Импорты

In [14]:
import os
import shutil
from PIL import Image
import pandas as pd
import random

### Константы

In [15]:
# путь к папке с исходными изображениями
DIR_INIT = 'data\\img_dir_clear'
# путь папки для тренировочной выборки
DIR_TRAIN = 'data\\img_dir_train'
# путь папки для валидационной выборки
DIR_VALID = 'data\\img_dir_valid'
# путь папки для тестовой выборки
DIR_TEST = 'data\\img_dir_test'
# Классы
CLASSES = ['0', '090', '180', '270']

### Собственные функции

In [16]:
def data_preparation_train(path_=''):
    """ Функция поворачивает каждый файл на 90, 180, 270 гр. и сохраняет в поддиректории с метками классов
    Args:
        path_ (str): путь к папке с файлами
    Returns:
        нет
    """
    # Формируем список файлов
    file_list_init = os.listdir(path_)

    # Повороты по 90 градусов, метки классов "090", "180", "270"
    for angle in ['090', '180', '270']:
        for file_name in file_list_init:
            if file_name.lower().endswith(('.png', '.jpg', '.jpeg')):
                # открываем изображение, поворачиваем на 90/180/270, сохраняем экземпляр
                file_name_new = angle + '_' + file_name
                file_path = os.path.join(path_, file_name)
                image = Image.open(file_path)
                image_rot = image.rotate(angle=int(angle), expand=True)
                image_rot.save(os.path.join(path_, str(angle), file_name_new))
                image.close()

    # Обрабатываем метку класса "0" гр.
    for file_name in file_list_init:
        if file_name.lower().endswith(('.png', '.jpg', '.jpeg')):
            # Перемещаем исходые файлы в папку с меткой "0"
            path_init = os.path.join(path_, file_name)
            path_dest = os.path.join(path_, '0', file_name)
            shutil.move(path_init, path_dest)

    # Выводим инфо по обработанному сету
    sum = 0
    print(f'Сет {path_} \nКоличество примеров по классам:')
    for i in CLASSES:
        print(i, '\t', len(os.listdir(os.path.join(path_, i))))
        sum += len(os.listdir(os.path.join(path_, i)))
    print(f'Всего примеров: ', sum)

In [17]:
def data_preparation_test(path_=''):
    """ Функция поворачивает каждый файл в директории на 90, 180, 270 гр. и сохраняет в поддиректории с метками классов
    Args:
        path_ (str): путь к папке с файлами
    Returns:
        нет
    """
    # Формируем список файлов
    file_list_init = os.listdir(path_)

    # Повороты по 90 градусов, метки классов "90", "180", "270"
    for angle in ['090', '180', '270']:
        for file_name in file_list_init:
            if file_name.lower().endswith(('.png', '.jpg', '.jpeg')):
                # открываем изображение, поворачиваем, сохраняем
                file_name_new = angle + '_' + file_name
                file_path = os.path.join(path_, file_name)
                image = Image.open(file_path)
                image_rot = image.rotate(angle=int(angle), expand=True)
                image_rot.save(os.path.join(path_, angle, file_name_new))
                image.close()

    # Перемещаем исходые файлы в папку с меткой "0"
    for file_name in file_list_init:
        if file_name.lower().endswith(('.png', '.jpg', '.jpeg')):
            path_init = os.path.join(path_, file_name)
            path_dest = os.path.join(path_, '0', file_name)
            shutil.move(path_init, path_dest)

    # Выводим инфо по обработанному сету
    sum = 0
    print(f'Сет {path_} \nКоличество примеров по классам:')
    for i in CLASSES:
        print(i, '\t', len(os.listdir(os.path.join(path_, i))))
        sum += len(os.listdir(os.path.join(path_, i)))
    print(f'Всего примеров: ', sum)

### Основной код

In [18]:
# Проверяем существование директорий для трейна, валидации и теста, создаем их, если не существуют.
for i in [DIR_INIT, DIR_TRAIN, DIR_VALID, DIR_TEST]:
    if not os.path.exists(i):
        os.makedirs(i)

In [19]:
# Cоздаем поддиректории для классов изображений
for i in CLASSES:
    os.makedirs(DIR_TRAIN + '\\' + i)
    os.makedirs(DIR_VALID + '\\' + i)
    os.makedirs(DIR_TEST + '\\' + i)

In [20]:
# Создаем списки файлов для тренировочной, валидационной и тестовой выборки в соотношении 64:16:20, соотв.
file_list = [f for f in os.listdir(DIR_INIT)
             if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

random.shuffle(file_list)
train_size = int(len(file_list) * 0.64)
val_size = int(len(file_list) * 0.16)
train_files = file_list[:train_size]
val_files = file_list[train_size:val_size+train_size]
test_files = file_list[val_size+train_size:]
print(f'Тренировочная выборка: {len(train_files)} \nВалидационная выборка: {len(val_files)} \nТестовая выборка: {len(test_files)}')

Тренировочная выборка: 110 
Валидационная выборка: 27 
Тестовая выборка: 36


In [21]:
# Копируем файлы трейна, валидации и теста в соответствующие папки
for fn in train_files:
    sourse_path = os.path.join(DIR_INIT, fn)
    dest_path = os.path.join(DIR_TRAIN, fn)
    shutil.copy(sourse_path, dest_path)

for fn in val_files:
    sourse_path = os.path.join(DIR_INIT, fn)
    dest_path = os.path.join(DIR_VALID, fn)
    shutil.copy(sourse_path, dest_path)

for fn in test_files:
    sourse_path = os.path.join(DIR_INIT, fn)
    dest_path = os.path.join(DIR_TEST, fn)
    shutil.copy(sourse_path, dest_path)

In [22]:
# Обработка тренировочного сета
data_preparation_train(path_=DIR_TRAIN)

Сет data\img_dir_train 
Количество примеров по классам:
0 	 110
090 	 110
180 	 110
270 	 110
Всего примеров:  440


In [23]:
# Обработка валидационного сета
data_preparation_test(path_=DIR_VALID)

Сет data\img_dir_valid 
Количество примеров по классам:
0 	 27
090 	 27
180 	 27
270 	 27
Всего примеров:  108


In [24]:
# Обработка тестовой выборки
data_preparation_test(path_=DIR_TEST)

Сет data\img_dir_test 
Количество примеров по классам:
0 	 36
090 	 36
180 	 36
270 	 36
Всего примеров:  144
